In [1]:
'''
tcloud中用销售订单生成税局发票模板
'''
import os
import re
import easygui
import openpyxl
import numpy as np
import pandas as pd
import shutil

In [2]:
def danjuChuli(df):
    df = df.dropna(how='all', axis=0)
    cols = [i for i in df.columns.to_list() if 'Unnamed' not in i]
    df = df[cols]
    # get maxrows
    for label, ser in df.items():
        for num, x in enumerate(ser):
            if isinstance(x, str):
                if '合计' in x:
                    index = num
                    print(index)
                    break
    df = df.iloc[:index]
    df = df.dropna(how='all', axis=1)
    return df
def chuliMingchen(d):
    d['存货名称'] = d['存货名称'].str.replace('运费',"")
    qian = d['存货名称'].str.split('型').str[0]
    hou = d['存货名称'].str.split('型').str[1]
    qian = pd.Series(qian).fillna('')
    hou = pd.Series(hou).fillna('')
    return d,qian,hou

In [3]:
#读取发票模板
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
df_fapiao = pd.read_excel(fname_fapiao,header = 2,dtype = {'商品和服务税收分类编码':'str'})
df_fapiao

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型


In [9]:
fname_dingdan = r"F:\a00nutstore\008\zw08\新公司\销货单明细表.xlsx"
df_dingdan = pd.read_excel(fname_dingdan,sheet_name = 0,header = 7)
df_dingdan

,Unnamed: 0,单据日期,创建时间,单据编号,业务类型,票据类型,客户编码,客户,部门编码,部门,...,赠品,费用分摊金额,累计出库数量,累计开票数量,累计开票含税金额,普通退货数量,冲抵退货数量,累计结款金额,销售订单号,明细备注
0,NaN,2024-09-06,2024-09-09 13:32:39,SA-2024-02-0961,普通销售,专用发票,040409-002,昆明市官渡区林林林文体用品经营部,010104-002,成都,...,否,NaN,800.0,NaN,NaN,NaN,NaN,NaN,SO-2024-02-0950,NaN
1,NaN,2024-09-06,2024-09-09 13:32:39,SA-2024-02-0961,普通销售,专用发票,040409-002,昆明市官渡区林林林文体用品经营部,010104-002,成都,...,否,NaN,1600.0,NaN,NaN,NaN,NaN,NaN,SO-2024-02-0950,NaN
2,NaN,2024-09-06,2024-09-09 13:32:38,SA-2024-02-0960,普通销售,专用发票,010188-002,湖北昇耀商贸有限公司,010101-002,汉办,...,否,NaN,540.0,NaN,NaN,NaN,NaN,NaN,SO-2024-02-0949,NaN
3,NaN,2024-09-06,2024-09-09 13:32:38,SA-2024-02-0960,普通销售,专用发票,010188-002,湖北昇耀商贸有限公司,010101-002,汉办,...,否,NaN,320.0,NaN,NaN,NaN,NaN,NaN,SO-2024-02-0949,NaN
4,NaN,2024-09-06,2024-09-09 13:32:38,SA-2024-02-0960,普通销售,专用发票,010188-002,湖北昇耀商贸有限公司,010101-002,汉办,...,否,NaN,200.0,NaN,NaN,NaN,NaN,NaN,SO-2024-02-0949,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,NaN,制表人:,张文伟,NaN,打印日期:,2024-09-29,NaN,其他信息:,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_dingdan1 = danjuChuli(df_dingdan) 
qian,hou = chuliMingchen(df_dingdan1)
df_dingdan1

5


,序号,货号,存货名称,件含量,数量（件）,数量（本）,单价,扣率（%）,金额（元）
0,1,N0681,A540型牛卡无线胶订本,320,3.0,960,0.87,0.97,810.14
1,2,N0516办,B540型牛卡无线胶装本,240,3.0,720,1.09,0.97,761.26
2,3,RL-1660-H01,16K60型卡面软抄,200,3.0,600,1.25,1,750.00
3,4,EWCGB5100,B5-100页无线胶装白皮草稿本,100,2.0,200,2.10,1,420.00
4,5,ATHA460B-萌宠派对,A460型精品卡面图画本,140,2.0,280,1.91,0.95,508.06


In [7]:
df_dingdan1['项目名称'] = df_dingdan1['货号'] + '-' + hou
df_dingdan1['项目名称'] = df_dingdan1['项目名称'].str.replace('运费-','运费')
df_dingdan1['商品和服务税收分类编码'] = '1060202010000000000'
df_dingdan1['规格型号'] = qian
df_dingdan1['单位'] = '本'
df_dingdan1['商品数量'] = df_dingdan1['数量（本）']
df_dingdan1['商品单价'] = df_dingdan1['单价']
df_dingdan1['金额'] = df_dingdan1['金额（元）']
df_dingdan1['税率'] = 0.01
df_dingdan1['折扣金额'] = ''
df_dingdan1['优惠政策类型'] = ''

df_dingdan2 = df_dingdan1[['项目名称',
 '商品和服务税收分类编码',
 '规格型号',
 '单位',
 '商品数量',
 '商品单价',
 '金额',
 '税率',
 '折扣金额',
 '优惠政策类型']]
df_dingdan2

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型
0,N0681-牛卡无线胶订本,1060202010000000000,A540,本,960,0.87,810.14,0.01,,
1,N0516办-牛卡无线胶装本,1060202010000000000,B540,本,720,1.09,761.26,0.01,,
2,RL-1660-H01-卡面软抄,1060202010000000000,16K60,本,600,1.25,750.00,0.01,,
3,EWCGB5100-,1060202010000000000,B5-100页无线胶装白皮草稿本,本,200,2.10,420.00,0.01,,
4,ATHA460B-萌宠派对-精品卡面图画本,1060202010000000000,A460,本,280,1.91,508.06,0.01,,


In [8]:
#提取发票台头等
fname_taitou = r"F:\a00nutstore\008\zw08\新公司\腾顺SaleOrder.xlsx"
df_taitou = pd.read_excel(fname_dingdan,sheet_name = 0)
df_taitou1 = df_taitou.iloc[:,[1,3]]
df_taitou1.columns = ['kehu','names' ]
for i,j in zip(df_taitou1['kehu'],df_taitou1['names']):
    if i == '发货单号：':
        danhao = j
    elif i == '客户名称：':
        kehu = j
    else:
        continue
print(danhao,kehu)

quan_jian = {'湖北腾顺商贸有限公司':'腾顺'}
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
path,file = os.path.split(fname_fapiao)
qian,suffix  = os.path.splitext(fname_fapiao)
gongsi = quan_jian.get('肥猫音树贸易（武汉）有限公司','待定')
newname = "".join([qian,'-',gongsi,danhao,suffix])
shutil.copyfile(fname_fapiao, newname)


with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
  
    df_dingdan2.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)

os.startfile(newname)






SO-2024-02-0944 湖北腾顺商贸有限公司


C:\Users\redda\.conda\envs\py311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [10]:
os.startfile(newname)